In [1]:
# essential packages
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data import random_split

# python modules
import matplotlib.pyplot as plt
import glob
import yaml
import numpy as np
import pandas as pd
import random

# custom module
from aux_code.learning_preprocess import convert_from_NED_to_Robotic, get_trajectory_before_pass

/home/mingi/anaconda3/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'libc10_cuda.so: cannot open shared object file: No such file or directory'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [131]:
from functools import partial

from ray import tune
from ray.air import Checkpoint, session
from ray.tune.schedulers import ASHAScheduler

## Data Loader

In [132]:
file_name = "aisdk_20180101"
howmany=200
using_file_indices = np.arange(0,howmany)

In [133]:
df_entire_pass = pd.DataFrame()

# https://www.w3resource.com/pandas/dataframe/dataframe-to_pickle.php
for using_file_idx in using_file_indices:
    unpickled_df = pd.read_pickle("./dk_csv_20180101/aisdk_20180101_{}.pkl".format(str(using_file_idx)))
    unpickled_df
    df_entire_pass = pd.concat([df_entire_pass, unpickled_df])

In [134]:
df_entire_pass

,x,y,v_x,v_y,rel_dist,rel_bearing,rel_bearing_diff,Heading,valid,obj_index,label
Timestamp,,,,,,,,,,,
2018-01-01 23:34:30,4.214136,2.391927,-0.009386,-0.006086,4.845643,0.516252,-0.001975,145.0,True,0,L
2018-01-01 23:34:45,4.086469,2.312937,-0.008511,-0.005266,4.695626,0.515043,-0.001209,145.5,True,0,L
2018-01-01 23:35:00,3.972708,2.235017,-0.007584,-0.005195,4.558257,0.512460,-0.002584,147.0,True,0,L
2018-01-01 23:35:15,3.857949,2.154808,-0.007651,-0.005347,4.418933,0.509374,-0.003086,150.5,True,0,L
2018-01-01 23:35:30,3.730337,2.078144,-0.008507,-0.005111,4.270140,0.508272,-0.001102,150.0,True,0,L
...,...,...,...,...,...,...,...,...,...,...,...
2018-01-01 11:37:00,-0.388895,0.516592,-0.002982,0.003150,0.646611,2.216093,0.012567,235.0,True,199,L
2018-01-01 11:37:15,-0.433163,0.564382,-0.002951,0.003186,0.711447,2.225403,0.009310,235.0,True,199,L
2018-01-01 11:37:30,-0.475801,0.610369,-0.002843,0.003066,0.773910,2.232930,0.007527,234.0,True,199,L


### pre-processing

In [136]:
df_entire_pass['heading_converted']  = np.deg2rad(df_entire_pass['Heading'])
# lambda function method: very fast! 
# https://stackoverflow.com/questions/71249186/applying-function-to-column-in-a-dataframe
df_entire_pass['heading_converted'] = df_entire_pass['heading_converted'].apply(convert_from_NED_to_Robotic)


In [137]:
df_entire_pass

,x,y,v_x,v_y,rel_dist,rel_bearing,rel_bearing_diff,Heading,valid,obj_index,label,heading_converted
Timestamp,,,,,,,,,,,,
2018-01-01 23:34:30,4.214136,2.391927,-0.009386,-0.006086,4.845643,0.516252,-0.001975,145.0,True,0,L,-0.959931
2018-01-01 23:34:45,4.086469,2.312937,-0.008511,-0.005266,4.695626,0.515043,-0.001209,145.5,True,0,L,-0.968658
2018-01-01 23:35:00,3.972708,2.235017,-0.007584,-0.005195,4.558257,0.512460,-0.002584,147.0,True,0,L,-0.994838
2018-01-01 23:35:15,3.857949,2.154808,-0.007651,-0.005347,4.418933,0.509374,-0.003086,150.5,True,0,L,-1.055924
2018-01-01 23:35:30,3.730337,2.078144,-0.008507,-0.005111,4.270140,0.508272,-0.001102,150.0,True,0,L,-1.047198
...,...,...,...,...,...,...,...,...,...,...,...,...
2018-01-01 11:37:00,-0.388895,0.516592,-0.002982,0.003150,0.646611,2.216093,0.012567,235.0,True,199,L,-2.530727
2018-01-01 11:37:15,-0.433163,0.564382,-0.002951,0.003186,0.711447,2.225403,0.009310,235.0,True,199,L,-2.530727
2018-01-01 11:37:30,-0.475801,0.610369,-0.002843,0.003066,0.773910,2.232930,0.007527,234.0,True,199,L,-2.513274


## Train, test data split

In [138]:
# https://www.geeksforgeeks.org/how-to-randomly-select-elements-of-an-array-with-numpy-in-python/

# np.random.seed(0) # right now for consistent data testing


# for train, test data
use_df = df_entire_pass.loc[(df_entire_pass.valid == True)] # to be used, valid df
unique_id = use_df.obj_index.unique()

# data split
train_data_size = int(len(unique_id) * 0.8)
test_data_size = len(unique_id) - train_data_size
print("train size: {} \n test size: {}".format(train_data_size, test_data_size))

# split obj indexes
train_obj_id = np.random.choice(unique_id, size = train_data_size, replace=False)
test_obj_id = np.setdiff1d(unique_id, train_obj_id)
train_obj_id

train size: 160 
 test size: 40


array([176,  86, 171, 194, 186, 179,  28, 172,  94, 146,  13, 178,  15,
       148,  12, 145, 182,  46,  45,  56,  66, 191,  23, 142, 134,   4,
       141, 163,  67, 152,  54, 107, 112, 116, 190, 168,  19,  70, 161,
       147,  68,  79,  30,  53, 193, 136,  44, 149, 101,  97,  21,  63,
         7,  75,  20,  64, 187,  85,  36, 164, 111,  55,   1,  48,  78,
        96,  76, 170, 104, 137, 115, 129, 127, 124,  22,  18, 199, 167,
        51,  73, 162,  71, 183,  60, 130,  37,  69,  72,   9,  88,  17,
        59, 126, 110, 128, 144,  11,  26, 156, 108, 177,   3,  77, 139,
        31, 109, 133, 122,  10, 113, 165, 123, 151,  25,  41,  34,  27,
        80, 118,  58,  40,  90, 160,  32, 103, 197, 184, 192, 157,  84,
       166,  81, 188,  14,  93, 153,  24, 195,  92, 169, 132,  47,  95,
       120,  29, 125, 198,  57, 173, 185, 121,  98,  33, 181,  83,  74,
       131, 196, 117,  39])

In [140]:
def check_dimension(x_data, y_data):
    if len(x_data) == len(y_data):
        print("Input, output dimension is same. good to go")
    else:
        raise ValueError("Input, output dimension is different. Double check!")

In [141]:
# train data
df_cropped_train = get_trajectory_before_pass(df_entire_pass, train_obj_id)
df_cropped_train

,x,y,v_x,v_y,rel_dist,rel_bearing,rel_bearing_diff,Heading,valid,obj_index,label,heading_converted
0,4.506382,-2.131028,-0.001686,0.004118,4.984853,-0.441726,0.008945,251.000000,True,176,L,-2.809980
1,4.481085,-2.069254,-0.001686,0.004118,4.935781,-0.432603,0.009123,250.000000,True,176,L,-2.792527
2,4.451802,-2.005076,-0.001952,0.004279,4.882506,-0.423184,0.009419,249.000000,True,176,L,-2.775074
3,4.434531,-1.968834,-0.001151,0.002416,4.851945,-0.417835,0.005349,248.000000,True,176,L,-2.757620
4,4.395439,-1.888308,-0.002606,0.005368,4.783889,-0.405766,0.012069,244.000000,True,176,L,-2.687807
...,...,...,...,...,...,...,...,...,...,...,...,...
35625,0.404765,-0.095161,-0.006227,0.000060,0.415800,-0.230908,-0.040407,203.974359,True,39,L,-1.989228
35626,0.311353,-0.094254,-0.006227,0.000060,0.325307,-0.293952,-0.063044,204.025641,True,39,L,-1.990123
35627,0.166101,-0.070444,-0.009683,0.001587,0.180422,-0.401112,-0.107159,204.076923,True,39,L,-1.991018
35628,0.086079,-0.050884,-0.005335,0.001304,0.099994,-0.533874,-0.132763,204.128205,True,39,L,-1.991913


In [142]:
np.all(df_cropped_train['label'])

True

In [143]:
np.any(np.isnan(df_cropped_train['y']))

False

In [144]:
# test data
df_cropped_test = get_trajectory_before_pass(df_entire_pass, test_obj_id)
df_cropped_test

,x,y,v_x,v_y,rel_dist,rel_bearing,rel_bearing_diff,Heading,valid,obj_index,label,heading_converted
0,4.214136,2.391927,-0.009386,-0.006086,4.845643,0.516252,-0.001975,145.0,True,0,L,-0.959931
1,4.086469,2.312937,-0.008511,-0.005266,4.695626,0.515043,-0.001209,145.5,True,0,L,-0.968658
2,3.972708,2.235017,-0.007584,-0.005195,4.558257,0.512460,-0.002584,147.0,True,0,L,-0.994838
3,3.857949,2.154808,-0.007651,-0.005347,4.418933,0.509374,-0.003086,150.5,True,0,L,-1.055924
4,3.730337,2.078144,-0.008507,-0.005111,4.270140,0.508272,-0.001102,150.0,True,0,L,-1.047198
...,...,...,...,...,...,...,...,...,...,...,...,...
9312,0.290117,0.043699,-0.002775,0.001358,0.293389,0.149501,0.079311,198.0,True,189,L,-1.884956
9313,0.250299,0.053381,-0.002654,0.000645,0.255928,0.210120,0.060619,198.0,True,189,L,-1.884956
9314,0.179070,0.068803,-0.004749,0.001028,0.191833,0.366834,0.156714,198.0,True,189,L,-1.884956
9315,0.122623,0.086667,-0.003763,0.001191,0.150158,0.615258,0.248425,197.0,True,189,L,-1.867502


## Train, test data onehot encoding and extract specific cell

In [145]:
# columns to extract
columns_for_x = ['x', 'y', 'rel_dist', 'rel_bearing', 'heading_converted']
# columns_for_x = ['r_value', 'atan']
columns_for_y = ['label']

# one hot encoding for y target
one_hot_lookup = torch.eye(2).tolist()

# df to array
X_train = []
X_test = []
y_train = []
y_test = []

#### train dataset
for obj_id, df_by_ID in df_cropped_train.groupby('obj_index'):
    X_train.append(df_by_ID[columns_for_x].values.tolist())
    # y_train.append(one_hot_lookup[0] if df_by_ID[columns_for_y].values[-1][0] == "L" else one_hot_lookup[1]) # last row, only char
    y_train.append(0 if df_by_ID[columns_for_y].values[-1][0] == "L" else 1) # last row, only char

    #### double check train dataset
check_dimension(X_train, y_train)

#### test dataset
for obj_id, df_by_ID in df_cropped_test.groupby('obj_index'):
    X_test.append(df_by_ID[columns_for_x].values.tolist())
    # y_test.append(one_hot_lookup[0] if df_by_ID[columns_for_y].values[-1][0] == "L" else one_hot_lookup[1]) # last row, only char
    y_test.append(0 if df_by_ID[columns_for_y].values[-1][0] == "L" else 1) # last row, only char

#### double check test dataset
check_dimension(X_test, y_test)

# y_train

Input, output dimension is same. good to go
Input, output dimension is same. good to go


## LSTM model

### Hyper parameters

In [146]:
# config = {
#     "hidden_size": tune.choice([2 ** i for i in range(7,9)]),
#     "num_layers": tune.choice([2* (i+1) for i in range(3,5)]),
#     "lr": tune.loguniform(1e-2, 1e-1),
#     "batch_size": tune.choice([16, 32]),
#     "step_size": tune.choice([50, 100]), 
#     "gamma": tune.choice([0.1* (i+1) for i in range(7, 9)])
    
# }

In [147]:
config = {
    "hidden_size": tune.choice([2 ** (i+1) for i in range(4,9)]),
    "num_layers": tune.choice([2* (i+1) for i in range(2,5)]),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16, 32]),
    "step_size": tune.choice([20, 30, 40, 50, 100, 150])
}

In [148]:
# config = {
#     "hidden_size": tune.grid_search([2 ** (i+1) for i in range(9)]),
#     "num_layers": tune.grid_search([2* (i+1) for i in range(5)]),
#     "lr": tune.loguniform(1e-4, 1e-1),
#     "batch_size": tune.grid_search([2, 4, 8, 16, 32]),
#     "step_size": tune.grid_search([20, 40, 100]), 
#     "gamma": tune.grid_search([0.1* (i+1) for i in range(5)])
# }

In [149]:
# sequence_length = 10 # number of timestamps # padding
input_size = 5 # number of columns, features
batch_size = config['batch_size'] # number of samples sent to the model at one time 32

hidden_size = config['hidden_size'] # dimension of hidden state # 500
num_layers = config['num_layers']  # total layer
num_classes = 1 # output class (L or R)
num_epochs = 500
learning_rate = config['lr']
clip=1 # gradient clipping


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [150]:
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

# Convert sequences to PyTorch tensors
# sequences: List of variable-length sequences
# targets: List of target labels

def pad_sequential_data(X_data, y_data):
    """
    Args:
        X_data: (list) training or test X data, sequential, not equal length
        y_data: (list) training or test y data

    Returns:
        - same length sequence for X_data

    reference: https://chat.openai.com/c/235f65e4-3a26-4418-a88c-ecf521cc5d8d
    """
    sequences = [torch.tensor(seq) for seq in X_data]
    targets = torch.tensor(y_data)

    #### Sort sequences by length in descending order
    # sequences[i] indicatess i th object (variable length)
    # sequence[i][0] (dimension: feature numbers)

    # sorted_indices = sorted(range(len(sequences)), key=lambda i: len(sequences[i][0]), reverse=True)
    sorted_indices = sorted(range(len(sequences)), key=lambda i: len(sequences[i]), reverse=True)

    #### sorted as per length
    sequences = [sequences[i] for i in sorted_indices]
    targets = targets[sorted_indices]

    #### Pad the sequences to make them the same length (zero padding as default)
    padded_sequences = pad_sequence(sequences, batch_first=True)
    return padded_sequences, targets

In [151]:
def load_data(X_train, y_train, X_test, y_test, batch_size):
    #### Create a TensorDataset from the padded sequences and targets
    padded_sequences, targets = pad_sequential_data(X_train, y_train)
    train_dataset = TensorDataset(padded_sequences, targets)
    # train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

    #### Getting train, validation data
    test_abs = int(len(train_dataset) * 0.8)
    train_subset, val_subset = random_split(train_dataset, [test_abs, len(train_dataset) - test_abs])

    train_loader = DataLoader(train_subset, batch_size=batch_size, shuffle=True, drop_last=True) # https://stackoverflow.com/a/53286859
    val_loader = DataLoader(val_subset, batch_size=batch_size, shuffle=True)

    #### Getting test data
    padded_sequences_test, targets_test = pad_sequential_data(X_test, y_test)
    test_dataset = TensorDataset(padded_sequences_test, targets_test)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True) # drop last didn't apply to test_loader to avoid null test_loader

    return train_loader, val_loader, test_loader


### LSTM 2D model with packed padding

In [152]:

class LSTM2D(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(LSTM2D, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.lstm = nn.LSTM(input_size, 
                            hidden_size, 
                            num_layers, 
                            batch_first=True,
                            dropout = 0.3)
        # self.batch_norm = nn.BatchNorm1d(hidden_size)
        self.fc = nn.Linear(hidden_size, num_classes)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, lengths):
        packed_sequences = pack_padded_sequence(x, lengths, batch_first=True, enforce_sorted=False)

        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        packed_out, _ = self.lstm(packed_sequences, (h0, c0))
        
        # Decode the hidden state of the last time step
        # out = self.batch_norm(out[:, -1, :])
        # out = self.fc(out)
        # // up to here batchnorm use
        out, _ = pad_packed_sequence(packed_out, batch_first=True)

        out = out[:, -1, :]
        # out = self.batch_norm(out)
        out = self.fc(out)
        # https://stackoverflow.com/questions/66456541/runtimeerror-cuda-error-device-side-assert-triggered-on-loss-function
        out = self.sigmoid(out) # ouput [0, 1]
        
        return out


### RNN define and initialization

In [153]:
# Training loop
loss_history =torch.tensor([])

def train_lstm(config):
    global loss_history

    model = LSTM2D(input_size, config["hidden_size"], config["num_layers"], num_classes).to(device)
    # print(model)

    criterion = nn.BCELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=config["lr"])
    # scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=config["step_size"], gamma=config["gamma"])
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=config["step_size"], gamma=0.1)

    model.train()

    train_loader, val_loader, test_loader = load_data(X_train, y_train, X_test, y_test, config["batch_size"])

    for epoch in range(num_epochs):
        ##################################################################
        ##### training 
        ##################################################################
        for batch_sequences, batch_labels in train_loader:
            optimizer.zero_grad()

            # 1) Calculate the sequence lengths for the current batch
            lengths = torch.sum(batch_sequences.sum(dim=2) != 0, dim=1)

            batch_sequences = batch_sequences.to(device)
            batch_labels = batch_labels.to(device)
            
            # 2) Forward pass
            output = model(batch_sequences, lengths)
            
            # this is possible, but I did instead output squeeze to match dimension
            # batch_labels = torch.unsqueeze(batch_labels, 1)

            # 3) Compute the loss       
            loss = criterion(output.squeeze(), batch_labels.float())

            # 4) Backward pass and optimization
            loss.backward()
            
            # 5) Gradient Clipping
            # 5-1) Gradient Norm Clipping
            # nn.utils.clip_grad_norm_(model.parameters(), max_norm=2.0, norm_type=2)
            nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
            # 5-2) Gradient Value Clipping
            # nn.utils.clip_grad_value_(model.parameters(), clip_value=1.0)
        
            optimizer.step()

        loss_history = torch.cat([loss_history, torch.tensor([loss.item()]).float()], dim=0)

        # Print the loss for every epoch
        # print(f"Epoch {epoch+1}/{num_epochs}, Training Loss: {loss.item()}")

        ##################################################################
        ##### Validation
        ##################################################################
        val_loss = 0.0
        val_steps = 0
        total_val = 0
        correct_val = 0
        for i, data in enumerate(val_loader, 0):
            with torch.no_grad():
                batch_sequences_val, batch_labels_val = data

                # 1) Calculate the sequence lengths for the current batch
                lengths_val = torch.sum(batch_sequences_val.sum(dim=2) != 0, dim=1)
                batch_sequences_val = batch_sequences_val.to(device)
                batch_labels_val = batch_labels_val.to(device)

                # 2) Forward pass
                output_val = model(batch_sequences_val, lengths_val)
                total_val += batch_labels_val.size(0)      

                ### prediction
                pred_val = torch.round(output_val.squeeze())  # rounds to the nearest integer
                #_, predicted = torch.max(output_val, 1)
                
                correct_tensor_val = pred_val.eq(batch_labels_val.float().view_as(pred_val))
                current_correct_val = np.squeeze(correct_tensor_val.numpy()) if not torch.cuda.is_available() else np.squeeze(correct_tensor_val.cpu().numpy())
                correct_val += np.sum(current_correct_val)

                loss_val = criterion(output_val.squeeze(), batch_labels_val.float())
                val_loss += loss_val.cpu().numpy()
                val_steps += 1

                # print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss}")

        # for hyperparameter tuning by Ray Tune
        checkpoint_data = {
            "epoch": epoch,
            "net_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
        }


        checkpoint = Checkpoint.from_dict(checkpoint_data)

        session.report(
            {"loss": val_loss / val_steps, "accuracy": correct_val / total_val},
            checkpoint=checkpoint,
        )
        
        ##################################################################
        ##### scheduler
        ##################################################################
        scheduler.step()

    # print("Finished Training")

In [154]:
gpus_per_trial = 1
num_samples = 20

tune_scheduler = ASHAScheduler(
    metric="loss",
    mode="min",
    max_t=num_epochs,
    grace_period=1,
    reduction_factor=2,
)

# ...
result = tune.run(
    train_lstm,
    resources_per_trial={"cpu": 8, "gpu": gpus_per_trial},
    config=config,
    num_samples=num_samples,
    scheduler=tune_scheduler)
# https://discuss.ray.io/t/tune-run-vs-tuner-fit/7041/5

best_trial = result.get_best_trial("loss", "min", "last")
print(f"Best trial config: {best_trial.config}")
print(f"Best trial final validation loss: {best_trial.last_result['loss']}")
print(f"Best trial final validation accuracy: {best_trial.last_result['accuracy']}")


2023-07-25 21:03:13,579	INFO tune.py:657 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


Trial name,accuracy,loss,should_checkpoint
train_lstm_32491_00000,1,0.615466,True
train_lstm_32491_00001,0.96875,0.1657,True
train_lstm_32491_00002,0.90625,0.735234,True
train_lstm_32491_00003,0.90625,0.714599,True
train_lstm_32491_00004,0.03125,0.694073,True
train_lstm_32491_00005,0.90625,0.667062,True
train_lstm_32491_00006,0.875,0.907084,True
train_lstm_32491_00007,0.96875,0.170289,True
train_lstm_32491_00008,0.96875,0.452939,True
train_lstm_32491_00009,0.90625,0.666047,True


2023-07-25 23:07:30,410	INFO tune.py:1148 -- Total run time: 7456.83 seconds (7456.78 seconds for the tuning loop).


Best trial config: {'hidden_size': 128, 'num_layers': 8, 'lr': 0.025625820080429105, 'batch_size': 2, 'step_size': 40}
Best trial final validation loss: 0.0975457183085382
Best trial final validation accuracy: 1.0


In [155]:
def test_accuracy(model, batch_size):
    # Test the model
    model.eval()

    train_loader, val_loader, test_loader = load_data(X_train, y_train, X_test, y_test, batch_size)

    with torch.no_grad():
        correct = 0
        total = 0
        y_true = []
        y_pred = []

        for batch_sequences, batch_labels in test_loader:
            # Data pre-processing
            lengths = torch.sum(batch_sequences.sum(dim=2) != 0, dim=1)
            batch_sequences = batch_sequences.to(device)
            batch_labels = batch_labels.to(device)

            # Forward pass
            output = model(batch_sequences, lengths)
            total += batch_labels.size(0)      

            ### prediction
            pred = torch.round(output.squeeze())  # rounds to the nearest integer
            #_, predicted = torch.max(outputs, 1)
            
            correct_tensor = pred.eq(batch_labels.float().view_as(pred))
            current_correct = np.squeeze(correct_tensor.numpy()) if not torch.cuda.is_available() else np.squeeze(correct_tensor.cpu().numpy())
            correct += np.sum(current_correct)
            
            y_true.extend(batch_labels.tolist())
            y_pred.extend(pred.tolist())

        print('Test Accuracy of the model on the test data: {} %'.format(100 * correct / total))

        # Calculate other evaluation metrics (precision, recall, F1-score)
    from sklearn.metrics import precision_score, recall_score, f1_score

    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    f1 = f1_score(y_true, y_pred, average='weighted')

    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"F1-score: {f1}")

In [156]:
best_trained_model = LSTM2D(input_size, best_trial.config["hidden_size"], best_trial.config["num_layers"], num_classes)
best_trained_model.to(device)

best_checkpoint = best_trial.checkpoint.to_air_checkpoint()
best_checkpoint_data = best_checkpoint.to_dict()

best_trained_model.load_state_dict(best_checkpoint_data["net_state_dict"])

### test accuracy
test_accuracy(best_trained_model, best_trial.config["batch_size"])

Test Accuracy of the model on the test data: 85.0 %
Precision: 0.7224999999999999
Recall: 0.85
F1-score: 0.7810810810810811


/home/mingi/anaconda3/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
